In [1]:
import os, sys 
import pandas as pd
import pickle as pk
from pathlib2 import Path
from evaluate import autoGrader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#import dataframe as df
#pip install pathlib2

fd = Path(os.getcwd())
fd_data = fd/'data'

# Step 1: getting the data
*load training data*

In [3]:
d_train = pd.read_csv(fd_data/'training_data.csv',parse_dates = ['t_ob'])  #parse_dates: parse as date，自动改为时间，不需要再用pd.to_datetime转换
d_train.head(1)

,t_ob,风向,PM10,PM2.5,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速
0,2020-03-01 00:10:00,338.0,32.7,23.4,25.0,31.35,3.74,38.23,28.54,27.04,0.98,0.76,1015.2,91.0,9.3,1.4


In [4]:
# 检查数据，很干净，没有缺失，不需要额外清洗
d_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11437 entries, 0 to 11436
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   t_ob    11437 non-null  datetime64[ns]
 1   风向      11437 non-null  float64       
 2   PM10    11437 non-null  float64       
 3   PM2.5   11437 non-null  float64       
 4   室内温度    11437 non-null  float64       
 5   室内湿度    11437 non-null  float64       
 6   SO2     11437 non-null  float64       
 7   O3      11437 non-null  float64       
 8   NOx     11437 non-null  float64       
 9   NO2     11437 non-null  float64       
 10  NO      11437 non-null  float64       
 11  CO      11437 non-null  float64       
 12  气压      11437 non-null  float64       
 13  大气湿度    11437 non-null  float64       
 14  大气温度    11437 non-null  float64       
 15  风速      11437 non-null  float64       
dtypes: datetime64[ns](1), float64(15)
memory usage: 1.4 MB


In [20]:
#import matplotlib.pyplot as plt
#x=d_train['t_ob']
#y=d_train['PM2.5']
#plt.plot(x,y)

In [4]:
# Convert the time 't_ob' to the time since 2020-03-01 00:00:00
# This variable was used to take account of the trend over time
from datetime import datetime, date
time_original = '2020-03-01 00:00:00'
time_original_1 = datetime.strptime(time_original,"%Y-%m-%d %H:%M:%S")
d_train['time'] = d_train['t_ob']-time_original_1 
d_train['duration']=d_train['time'].astype('timedelta64[s]')/60

In [6]:
d_train.head(1)

,t_ob,风向,PM10,PM2.5,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,time,duration
0,2020-03-01 00:10:00,338.0,32.7,23.4,25.0,31.35,3.74,38.23,28.54,27.04,0.98,0.76,1015.2,91.0,9.3,1.4,0 days 00:10:00,10.0


In [24]:
d_train.corr()

,风向,PM10,PM2.5,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,duration
风向,1.000000,0.006869,0.025860,-0.087779,0.108137,0.138783,0.098514,-0.043319,-0.076065,0.088882,0.198285,-0.226646,-0.047083,0.150992,0.087857,0.061140
PM10,0.006869,1.000000,0.828606,0.138276,-0.163246,0.527362,0.057018,0.436437,0.470015,0.127349,0.392112,-0.002375,-0.250987,0.134616,-0.086153,0.047587
PM2.5,0.025860,0.828606,1.000000,0.065903,0.051086,0.406424,-0.059772,0.423575,0.443626,0.166886,0.546273,0.003758,0.074167,0.015691,-0.223474,-0.037758
室内温度,-0.087779,0.138276,0.065903,1.000000,0.213570,0.265436,0.100132,0.186924,0.227100,-0.034484,-0.038728,-0.437545,-0.077902,0.468843,-0.049331,0.457950
室内湿度,0.108137,-0.163246,0.051086,0.213570,1.000000,-0.070110,0.042318,-0.081266,-0.100437,0.020884,0.314790,-0.700927,0.501949,0.522099,-0.025879,0.594780
SO2,0.138783,0.527362,0.406424,0.265436,-0.070110,1.000000,0.126625,0.420186,0.429361,0.202512,0.339423,-0.154747,-0.371687,0.326764,0.006135,0.204087
O3,0.098514,0.057018,-0.059772,0.100132,0.042318,0.126625,1.000000,-0.470714,-0.469510,-0.266523,-0.175111,-0.337163,-0.601084,0.642521,0.325676,0.375715
NOx,-0.043319,0.436437,0.423575,0.186924,-0.081266,0.420186,-0.470714,1.000000,0.975084,0.643328,0.376245,0.114371,0.053623,-0.115623,-0.260202,-0.113100
NO2,-0.076065,0.470015,0.443626,0.227100,-0.100437,0.429361,-0.469510,0.975084,1.000000,0.457464,0.355513,0.102672,0.045740,-0.114286,-0.280706,-0.093434
NO,0.088882,0.127349,0.166886,-0.034484,0.020884,0.202512,-0.266523,0.643328,0.457464,1.000000,0.281260,0.104118,0.057110,-0.069063,-0.074253,-0.130919


*load features for closed test data*

In [5]:
X_ind_test = pd.read_csv(fd_data/'closed_test_X.csv', parse_dates = ['t_ob'])
X_ind_test.head(2)

,t_ob,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速
0,2020-06-01 00:00:00,129.500000,50.266667,26.851667,52.270000,5.965000,180.263333,7.696667,6.688333,0.656667,0.653333,1006.800000,76.333333,24.916667,1.483333
1,2020-06-01 00:10:00,134.333333,49.266667,26.870000,54.566667,6.213333,181.400000,7.388333,6.356667,0.671667,0.640000,1006.716667,74.866667,24.883333,1.466667


In [6]:
X_ind_test['time'] = X_ind_test['t_ob']-time_original_1
X_ind_test['duration']=X_ind_test['time'].astype('timedelta64[s]')/60

*build model*

# Step 2: create X,Y and train-test split

In [32]:
# random forest 有超参数，先试一下添加validation set的方法
# 70% train,15% validation,15% test  
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.3, random_state=1)
X_val_rf, X_test_rf, y_val_rf, y_test_rf = train_test_split(X_test_rf, y_test_rf, test_size=0.5, random_state=1)

In [33]:
print(X_train_rf.shape,X_val_rf.shape,X_test_rf.shape)

(8005, 15) (1716, 15) (1716, 15)


# Step 3: build a more complex model

In [ ]:
# random forest
# 1.import the model
from sklearn.ensemble import RandomForestRegressor
# 2.initialize the random forest model
rf = RandomForestRegressor(n_estimators=100)  #n_estimators: hyper-parameter, represent the number of trees, default=100,test it

In [34]:
# 3.fit the model to the training set
model_rf=rf.fit(X_train_rf,y_train_rf)

In [35]:
# 4.evaluate the model performance
# R2 for random forest is much higher than linear regression
rf.score(X_train_rf, y_train_rf)

0.9986568618500528

In [36]:
rf.score(X_val_rf, y_val_rf)

0.9906280942744059

In [37]:
# test n_estimators as 80,120 manually
rf2 = RandomForestRegressor(n_estimators=80)
# fit the model to the training set
model_rf2=rf2.fit(X_train_rf,y_train_rf)
print(rf2.score(X_train_rf, y_train_rf),rf2.score(X_val_rf, y_val_rf))

0.998634422224369 0.9905824193526855


In [38]:
rf2 = RandomForestRegressor(n_estimators=120)
# fit the model to the training set
model_rf2=rf2.fit(X_train_rf,y_train_rf)
print(rf2.score(X_train_rf, y_train_rf),rf2.score(X_val_rf, y_val_rf))

0.9987506163280425 0.9907491423001845


# Step 4: hyperparameter tuning

###### method 1: using training set and validation set

In [39]:
import time

In [40]:
# hyper-parameter tuning
# number of trees to tune, set the potential value
start=time.time()
no_trees = [40,60,80,100,120,140,160,180,200,220,240,260,280,300]

# process of tune the number of trees
max_score = 0
best_no_trees = None
for t in no_trees:
    m = RandomForestRegressor(n_estimators=t).fit(X_train_rf,y_train_rf)  #train the model on the training set
    score = m.score(X_val_rf, y_val_rf)   # evaluate how the model performs on the validation set
    if score > max_score:
        max_score = score   # replace the former one if it performs better
        best_no_trees = t  
end=time.time()

In [44]:
?time.time

In [41]:
# time for running the code
print(end-start)

157.53649711608887


In [45]:
# get the best result, r2 for validation set improved
print(max_score,best_no_trees)  

0.9909398332497187 200


In [46]:
rf_final = RandomForestRegressor(n_estimators = best_no_trees) #把通过上面测试得到的最优值带进去
rf_final.fit(X_train_rf, y_train_rf) #retrain the model

RandomForestRegressor(n_estimators=200)

In [47]:
print('The score on the training data:')
print(rf_final.score(X_train_rf, y_train_rf))
print('The score on the validation data:')
print(rf_final.score(X_val_rf, y_val_rf))

The score on the training data:
0.9987520278641333
The score on the validation data:
0.9908117393911386


###### method 2: cross validation

In [48]:
from sklearn import model_selection
start_cv=time.time()
# values of n_estimators and max_depth
parameters = {'n_estimators':[40,60,80,100,120,140,160,180,200,220,240,260,280,300]}
              #'max_depth':[10,20,30,40,50]}

# initialize the random forest model
rf = RandomForestRegressor()

#Exhaustive search over specified parameter values for an estimator
clf = model_selection.GridSearchCV(rf, parameters)

# fit the model on the training set
clf.fit(X_train_rf, y_train_rf)

# we can query the best parameter value and its accuracy score
print ("The best parameter value is: ")
print (clf.best_params_)
print ("The best score is: ")
print (clf.best_score_)
end_cv=time.time()

The best parameter value is: 
{'n_estimators': 240}
The best score is: 
0.9886951014602609


In [49]:
print(end_cv-start_cv)

627.0394279956818


In [50]:
clf.best_params_['n_estimators']

240

In [51]:
rf_final_cv = RandomForestRegressor(n_estimators = clf.best_params_['n_estimators']) #把通过上面测试得到的最优值带进去
rf_final_cv.fit(X_train_rf, y_train_rf) 

RandomForestRegressor(n_estimators=240)

In [52]:
# 添加validation set获得的结果
print('The score on the training data:')
print(rf_final.score(X_train_rf, y_train_rf))
print('The score on the validation data:')
print(rf_final.score(X_val_rf, y_val_rf))

The score on the training data:
0.9987520278641333
The score on the validation data:
0.9908117393911386


In [53]:
# cross validation获得的结果
print('The score on the training data:')
print(rf_final_cv.score(X_train_rf, y_train_rf))
print('The score on the validation data:')
print(rf_final_cv.score(X_val_rf, y_val_rf))

The score on the training data:
0.9988096946466758
The score on the validation data:
0.9907041876804705


# Step 5: use the final model for prediction

In [54]:
# get final estimate
rf_final.score(X_test_rf, y_test_rf)

0.9912436713689449

In [63]:
# x for prediction
x_for_prediction=X_ind_test.drop(['t_ob','time'], axis=1) 
x_for_prediction.head(1)

,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,duration
0,129.5,50.266667,26.851667,52.27,5.965,180.263333,7.696667,6.688333,0.656667,0.653333,1006.8,76.333333,24.916667,1.483333,132480.0


In [64]:
X_ind_test['prediction'] = rf_final.predict(x_for_prediction)

In [65]:
X_ind_test['prediction'] = rf_final.predict(x_for_prediction)
result=X_ind_test[['t_ob','prediction']] #提取出需要的列
result = result.rename(columns={ 'prediction': 'PM2.5'})  #修改列名
result.to_csv('submission_rf.csv',index=False)  #保存文件

In [66]:
# R2 improved a little bit
# max residual is big, outlier?
# try to tune more hyper-parameters, like max_depth
autoGrader(fd/'submission_rf.csv')

[Enter] to confirm your submission file is placed at D:\code_own\CodingTest-SunyuWang\to_players\submission_rf.csv)

Submission time: 2022-05-13 08:30:42.508629
 - RMSE: 4.877
 - R sq.: 0.839
 - Max. abs. residual: 26.675
 - Min. abs. residual: 0.003
 - Logging results. [Enter] to make sure 'submission_history.csv' is closed: 
 - Notes for this submission: 
 ! Results logged in 'submission_history.csv'.
 ! Submission file saved to '/old_submissions/submission_2022_05_13_08_30_42.csv'.
Evaluation Successful. Press to quit...
